# Query Sentinel 1 scene from SHUB and load it with stackstac
- Query shub
- Get aws url
- Convert to stack Item
- Load with stackstac

In [ ]:
from sentinelhub import CRS, BBox, DataCollection, SHConfig
from oxeo.core.data import get_aoi_from_s1_shub_catalog

from sentinelhub import SHConfig

config = SHConfig()
config

In [ ]:
from sentinelhub import SentinelHubCatalog

catalog = SentinelHubCatalog(config=config)


In [ ]:
caspian_sea_bbox = BBox([49.9604, 44.7176, 51.0481, 45.2324], crs=CRS.WGS84)
time_interval = "2020-12-10", "2021-02-01"

search_params = {
 "bbox": caspian_sea_bbox,
 "time": time_interval
}

aoi = get_aoi_from_s1_shub_catalog(catalog,search_params)



# Query Sentinel 1 scene from ASF and load it with stackstac
- Query asf
- Convert asf granule to aws url
- Convert to stack Item
- Load with stackstac

In [ ]:
%load_ext autoreload

%autoreload 2


import requests
import os
from oxeo.core.data import query_asf, asf_granule_to_aws, CusotomSentinel1Reader
from oxeo.core.stac import sentinel1
import stackstac
os.environ["AWS_REQUEST_PAYER"] = "requester"

In [ ]:
item = sentinel1.create_item(results[0]["assets"]["s3"]["href"])

In [ ]:
stack = stackstac.stack(item)
stack

In [ ]:
import pyproj
bbox = [49.9604, 44.7176, 51.0481, 45.2324]

min_x_utm, min_y_utm = pyproj.Proj(stack.crs)(
        bbox[0], bbox[1]
    )
max_x_utm, max_y_utm = pyproj.Proj(stack.crs)(
        bbox[2], bbox[3]
    )

In [ ]:
aoi = stack.loc[..., max_y_utm:min_y_utm, min_x_utm:max_x_utm]
aoi

In [ ]:
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

geo: dict = {'type': 'Polygon',
   'coordinates': [
          [
            [
              -3.14208984375,
              37.470498470798724
            ],
            [
              -2.7960205078125,
              37.330856613297144
            ],
            [
              -2.61474609375,
              37.49229399862877
            ],
            [
              -2.6751708984375,
              37.6968609874419
            ],
            [
              -3.0267333984375,
              37.640334898059486
            ],
            [
              -3.14208984375,
              37.470498470798724
            ]
          ]]}
polygon = shape(geo)

In [ ]:
df = query_asf(platform="Sentinel-1A,Sentinel-1B",
               processing_level="GRD_HD",
               beam_mode="IW",
               intersects_with_wkt=polygon.wkt,
               start="2020-04-05T00:00:00UTC",
               end="2020-05-05T00:00:00UTC")

In [ ]:
asf_granule_to_aws(df.iloc[0])

In [ ]:
s3_url = asf_granule_to_aws(df.iloc[0])
item = sentinel1.create_item(s3_url)

In [ ]:

arr = stackstac.stack(item,reader=CusotomSentinel1Reader)